In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import pickle
import csv

# Extract information from MIT-BIH raw Files

## Get Record Names from the RECORDS File

In [2]:
rlist = []
records = 'mit-bih-raw/RECORDS'
with open(records) as rfile:
    for record in rfile:
        record = record[0:len(record)-1]
        rlist.append(record)

## Extract all info using the WaveForm DataBase (WFDB)

In [3]:

###### Step 1: Initialize all Arrays
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist):
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb.rdsamp(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
        ######################################################
        samples.append(samp) #add it to our array for all subject
        
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list
        

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [01:15<00:00,  3.02s/it]


### Extracting Rhythm Data

Next, I am going to reformat the rhythm annotations into a different format that is more understandable.

The current format for an individuals rhythm annotations are as follows:
- `labels = ['(N', '(AFIB', '(N', '(O', ...]`
- `locs   = [  10,    1000, 1234, 1983, ...]`

Where the labels' corresponding locations are where that rhythm begins.

The below code changes it to the following format instead using the python data type Dictionary.

```python
rhythm_annotations = {
    '(N':    [ [10,   999],
               [1234, 1982]
             ], 
    '(AFIB': [ [1000, 1233]
             ],
    '(O':    [ [1983, ...]
             ]    
}
```


This data is now formatted in a 2-Dimensional array in which each pair of values represents a range of values in which a specific rythm is present. 

The data can be accessed like so: 
```python
  rhythm_annotations['(N']         = [ [10,   999],
                                       [1234, 1982]
                                     ]
    
  rhythm_annotations['(N'][0]      = [10,   999]

  rhythm_annotations['(N'][0][0]   = 10

```

In [4]:
atr_dics = [] #Initialize the array that will hold the dictionary for each subject

for idxs,lab in enumerate(atr_label):
    atr_dic = {} #Initialize dictionary for each subject
    for idx,x in enumerate(lab):
        if x not in atr_dic.keys():
            atr_dic[x] = [] #Add dictionary key if does not exist
        atr_dic[x].append([atr_locs[idxs][idx], atr_locs[idxs][idx+1]]) #Insert range for each rhythm
    atr_dics.append(atr_dic) #Add to dictionary array

## Other Ways To Format

Here we are going to format each subjects data into a DataFrame using pandas. Many statistical tools are built to be used with DataFrames. 

Also, it allows for a one-stop shop for our data where we can save all data for each subject in one file instead of having multiple files per subject.

Our data frame is going to be formatted like so:

|     | Signal 1 | Signal 2 | R-Peak | Normal | AFIB  | Other |
|-----|----------|----------|--------|--------|-------|-------|
| ... | ...      | ...      | ...    | ...    | ...   | ...   |
| 234 | 0.123    | -0.312   | True   | True   | False | False |
| ... | ...      | ...      | ...    | ...    | ...   | ...   |


- Column 1: Index
    - the index is the value of each row and represents the sample value
- Column 2: Signal 1
    - a float (or decimal) value which represents the value of the first signal in the reading at the given sample value
- Column 3: Signal 2
    - a float (or decimal) value which represents the value of the second signal in the reading at the given sample value
- Column 4: R-Peak
    - a boolean value (```True``` or ```False```) which represents if there is a R-Peak at the given sample value
- Column 5: Normal
    - a boolean value (```True``` or ```False```) which represents if the sample is in a pattern of Normal beats
- Column 6: AFIB
    - a boolean value (```True``` or ```False```) which represents if the sample is in a pattern of AFIB beats
- Column 7: Other
    - a boolean value (```True``` or ```False```) which represents if the sample is in a pattern of other beats

In [5]:
subject_dataframes = [] # Initialize the subject_dataframes - will hold all of our subject dataframes

for s, _ in enumerate(tqdm(good_list)): # Iterate through all of the subjects that we have complete data of 
    subj = pd.DataFrame( # The below statements initialize our datafram. The first to columns will be our given signals, and the rest we initialize to 0
        data = np.transpose(np.array([ # First we give our data, for pandas they want the data by row instead of by column, so we use transpose to get the proper format
                                               [x[0] for x in samples[s][0]],
                                               [x[1] for x in samples[s][0]],
                                                np.zeros(len(samples[s][0])), # np.zeros makes an array of zeros with the given lenth
                                               np.zeros(len(samples[s][0])), 
                                               np.zeros(len(samples[s][0])), 
                                               np.zeros(len(samples[s][0])), 
                                        ])
                           ),
        columns = ['Signal 1', 'Signal 2', 'R-Peak', 'Normal', 'AFIB', 'Other'] # Here we name our columns to match the dataframe we outlined above
    )
    norm = [] # Initialize the norm array which will list every index the person is in a normal rhythm
    if '(N' in atr_dics[s].keys():
        for x in atr_dics[s]['(N']: # Then we iterate through our ranges we extracted above
            norm = norm + list(range(x[0], x[1])) # And add all values in the range to our norm array
    af = [] # Then we do the same steps above for AFIB rhythms
    if '(AFIB' in atr_dics[s].keys():
        for x in atr_dics[s]['(AFIB']:
            af = af + list(range(x[0], x[1]))
    subj['R-Peak']= subj.index.isin(qrs[s]) # the isin() function of a DataFram index will return true if the index is in that list and false if it is not
                                            # then, we can initialize our dataFrame with correct values based on that
    subj['Normal']= subj.index.isin(norm)
    subj['AFIB'] = subj.index.isin(af)
    subj['Other'] = ~subj.index.isin(np.append(norm, af)) # Because we are classifying AFIB specifically we define other as any rhythm not in the norm or AFIB list

    
    subject_dataframes.append(subj) # Add the dataframe we built to our to array that holds all of our subjects' dataframes

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [09:22<00:00, 24.44s/it]


In [1]:
subject_dataframes[1]

NameError: name 'subject_dataframes' is not defined

## Saving Extracted Information 

Saving the information that we have used processing power to extract is important because:
1. It makes our data easier to access in the future
    - Easy access in new files
2. It creates static information for us to use and reference
3. By saving in in a CSV we make it more accessible for others to use
    - The data can now be used in an excel sheet and more
    
We will generally always extract to a CSV file unless the data is too complex. If that is the case then we have another option. 

'pickle' is a Python package which will save much more complex Data types for future use. 

For example - if you have want to save a statistical model, pickle will be able to do that more effectively than CSV format.

In [7]:
reload_flag = False
  # Set this flag to true to re-save all of the extracted information even if it has already been saved at these paths

In [8]:
for idx, x in enumerate(tqdm(good_list)): 
    if not os.path.exists('mit-bih-dataframes/'+x+ '.csv') or reload_flag:
        subject_dataframes[idx].to_csv('mit-bih-dataframes/'+x+'.csv') # Pandas DataFrames have a built in to_csv() function which whill save it at the passed path

np.savetxt("mit-bih-dataframes/subject_list.csv", good_list, delimiter=",",  fmt='%s') 
   # We'll load the complete list of subjects as well so that we can easily recreate the file names

100%|████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 7688.00it/s]


In [9]:
np.savetxt("mit-bih-extracted/subject_list.csv", good_list, delimiter=",",  fmt='%s') #Save the names in the folder 
for idx, x in enumerate(tqdm(good_list)): # Iterate through our subjects
    if not os.path.exists("mit-bih-extracted/"+x+"_signals.csv") or reload_flag:
        np.savetxt("mit-bih-extracted/"+x+"_signals.csv", np.array(samples[idx][0]), delimiter=",") # numPy has a savetxt() function which by setting the delimiter as ',' we can 
                                                                                            # simulate a to_csv() function 
    if not os.path.exists("mit-bih-extracted/"+x+"_rpeaks.csv") or reload_flag:
            np.savetxt("mit-bih-extracted/"+x+"_rpeaks.csv", np.array(qrs[idx]), delimiter=",")      
    if not os.path.exists("mit-bih-extracted/"+x+"_headers.pkl") or reload_flag:
        with open("mit-bih-extracted/"+x+"_headers.pkl", 'wb') as picklefile: # nomPy has no way to save a dictionary as a CSV so we use the pickle package
                                    # First we open up the file we would like to write to
            pickle.dump(samples[idx][1], picklefile)
    if not os.path.exists("mit-bih-extracted/"+x+"_rhythms.pkl") or reload_flag:
        with open("mit-bih-extracted/"+x+"_rhythms.pkl", 'wb') as picklefile:
            pickle.dump(atr_dics[idx], picklefile)


100%|████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 2091.20it/s]


In [ ]:
[12, 156, 299, 420, ....]

In [ ]:
[156-12, 299-156, 420-299, ...]